## Naver Shopping Reviews Sentiment Analysis
- GRU
- Mecab

In [3]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

url = 'https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/naver_shopping.txt'
df = pd.read_table(url, names=['ratings','reviews'])
df.head()

,ratings,reviews
0,5,배공빠르고 굿
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ


In [4]:
# 평점이 4, 5점인 데이터를 1(긍정), 나머지는 0(부정)
df['label'] = df.ratings.apply(lambda x: 1 if x >= 4 else 0)
df.head()

,ratings,reviews,label
0,5,배공빠르고 굿,1
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고,0
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...,1
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...,0
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ,1


In [5]:
df.label.value_counts()

0    100037
1     99963
Name: label, dtype: int64

In [6]:
df.shape

(200000, 3)

#### Data Preprocessing

In [7]:
# Null data 확인
df.isna().sum().sum()

0

In [8]:
# 중복데이터 확인
df.reviews.nunique()

199908

In [9]:
# 중복 제거
df.drop_duplicates(subset=['reviews'], inplace=True)
df.shape

(199908, 3)

In [10]:

df.reviews = df.reviews.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '')

C:\Users\lucky\AppData\Local\Temp\ipykernel_26300\1236495415.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df.reviews = df.reviews.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '')


In [11]:
# Null 데이터가 생기면 제거
df.reviews.replace('', np.nan, inplace=True)
df.isna().sum().sum()

0

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df.reviews.values, df.label.values, stratify=df.label.values,
    test_size=0.2, random_state=2022
)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((159926,), (39982,), (159926,), (39982,))

#### Tokenizer

In [18]:
from eunjeon import Mecab
mecab = Mecab()

In [19]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','을','ㅋㅋ','ㅠㅠ','ㅎㅎ']

In [20]:
from tqdm.notebook import tqdm

train_data = []
for sentence in tqdm(X_train):
    morphs = mecab.morphs(sentence)
    tmp_X = [word for word in morphs if word not in stopwords]
    train_data.append(tmp_X)

  0%|          | 0/159926 [00:00<?, ?it/s]

In [21]:
test_data = []
for sentence in tqdm(X_test):
    morphs = mecab.morphs(sentence)
    tmp_X = [word for word in morphs if word not in stopwords]
    test_data.append(tmp_X)

  0%|          | 0/39982 [00:00<?, ?it/s]

In [22]:
train_data[0]

['재', '구매', '늘', '먹', '던', '거', '예요', '밥맛', '좋', '아요']

In [23]:
import numpy as np
import tensorflow as tf

seed = 2022
np.random.seed(seed)
tf.random.set_seed(seed)

In [24]:
from tensorflow.keras.preprocessing.text import Tokenizer
t = Tokenizer()
t.fit_on_texts(train_data)

In [25]:
len(t.word_index)

40940

In [26]:
# 등장 빈도가 3 미만인 것의 갯수
threshold = 3
total_cnt = len(t.word_index)   # 308969
rare_cnt = 0        # 등장 빈도가 threshold 보다 작은 단어의 갯수
total_freq = 0      # 훈련 데이터의 전체 단어의 빈도수의 합
rare_freq = 0       # 등장 빈도가 threshold 보다 작은 단어의 등장 빈도수의 합

In [27]:
for key, value in t.word_counts.items():
    total_freq += value
    if value < threshold:
        rare_cnt += 1
        rare_freq += value

In [28]:
print('단어 집합(vocabulary)의 크기 :', total_cnt)
print(f'등장 빈도가 {threshold - 1}번 이하인 희귀 단어의 수: {rare_cnt}')
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 40940
등장 빈도가 2번 이하인 희귀 단어의 수: 23515
단어 집합에서 희귀 단어의 비율: 57.437713727405956
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 1.0875672342975908


In [29]:
# 0번 패딩 토큰, 1번 OOV(Out-of-value) 토큰을 고려하여 +2
vocab_size = total_cnt - rare_cnt + 2
vocab_size

17427

In [30]:
t = Tokenizer(num_words=vocab_size, oov_token='OOV')
t.fit_on_texts(train_data)
X_train = t.texts_to_sequences(train_data)
X_test = t.texts_to_sequences(test_data)

In [31]:
# 데이터의 최대/평균 길이
max(len(s) for s in X_train), sum(map(len, X_train)) / len(X_train)

(86, 16.348504933531757)

In [32]:
# 리뷰 길이를 60으로 설정하고 패딩
max_len = 60

In [33]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [34]:
X_train.shape, X_test.shape

((159926, 60), (39982, 60))

#### Modeling

In [35]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, GRU, Dense
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

model = Sequential([ 
    Embedding(vocab_size, 100, input_length=max_len),
    GRU(128),
    Dense(1, activation='sigmoid')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 60, 100)           1742700   
                                                                 
 gru (GRU)                   (None, 128)               88320     
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,831,149
Trainable params: 1,831,149
Non-trainable params: 0
_________________________________________________________________


In [36]:
model.compile('adam', 'binary_crossentropy', ['accuracy'])
model_path = './models/best-shopping-gru.h5'
mc = ModelCheckpoint(model_path, verbose=1, save_best_only=True)
es = EarlyStopping(patience=3)

In [37]:
hist = model.fit(
    X_train, y_train, validation_split=0.2,
    epochs=30, batch_size=128, callbacks=[mc, es]
)

Epoch 1/30
 999/1000 [============================>.] - ETA: 0s - loss: 0.2887 - accuracy: 0.8875
Epoch 00001: val_loss improved from inf to 0.24104, saving model to ./models\best-shopping-gru.h5
1000/1000 [==============================] - 13s 10ms/step - loss: 0.2887 - accuracy: 0.8876 - val_loss: 0.2410 - val_accuracy: 0.9107
Epoch 2/30
 997/1000 [============================>.] - ETA: 0s - loss: 0.2105 - accuracy: 0.9250
Epoch 00002: val_loss improved from 0.24104 to 0.22683, saving model to ./models\best-shopping-gru.h5
1000/1000 [==============================] - 10s 10ms/step - loss: 0.2105 - accuracy: 0.9251 - val_loss: 0.2268 - val_accuracy: 0.9169
Epoch 3/30
 996/1000 [============================>.] - ETA: 0s - loss: 0.1775 - accuracy: 0.9380
Epoch 00003: val_loss did not improve from 0.22683
1000/1000 [==============================] - 10s 10ms/step - loss: 0.1774 - accuracy: 0.9380 - val_loss: 0.2428 - val_accuracy: 0.9116
Epoch 4/30
 998/1000 [============================

In [38]:
best_model = load_model(model_path)
best_model.evaluate(X_test, y_test)

1250/1250 [==============================] - 4s 3ms/step - loss: 0.2325 - accuracy: 0.9137


[0.23250068724155426, 0.9136861562728882]

In [39]:
def sentiment_predict(review, tokenizer=t, max_len=max_len):
    review = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣]',' ',review).strip()
    morphs = mecab.morphs(review)
    morphs = [word for word in morphs if word not in stopwords]
    encoded = tokenizer.texts_to_sequences([morphs])
    padded = pad_sequences(encoded, maxlen=max_len)
    score = float(best_model.predict(padded))
    return f'긍정({score*100:.2f}%)' if score > 0.5 else f'부정({(1-score)*100:.2f}%)'

In [40]:
sentiment_predict('이 상품 진짜 좋아요... 저는 강추합니다. 대박')

'긍정(95.65%)'

In [41]:

sentiment_predict('진짜 배송도 늦고 개짜증나네요. 뭐 이런 걸 상품이라고 만듬?')

'부정(98.98%)'

In [42]:

sentiment_predict('판매자님... 너무 짱이에요.. 대박나삼')

'긍정(91.86%)'

In [43]:
sentiment_predict('ㅁㄴㅇㄻㄴㅇㄻㄴㅇ리뷰쓰기도 귀찮아')

'부정(82.63%)'